In [1]:
#-*- coding:utf-8 -*-

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC
import time, json
import openpyxl
from openpyxl.styles import Border, Side, PatternFill

# todo: 포스트 주소 리스트 불러오기
# todo: 포스트 멤버 리스트 불러오기
# todo: 검증 대상 멤버 불러오기
# todo: 비교 후 공감 빠드린 포스트 주소 체크
# todo: 비교 후 공감 빠뜨린 수 체크

In [2]:
options = Options()
options.page_load_strategy = 'eager'
driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
wait = WebDriverWait(driver, 10)

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [C:\Users\CY\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache


In [455]:
print(driver)


<selenium.webdriver.chrome.webdriver.WebDriver (session="d48f1b4e36d14c7ae1dce283e79eb98d")>


In [442]:
# # 교집합
# lst1 = ['A', 'B', 'C', 'D']
# lst2 = ['C', 'D', 'E', 'F']
# intersection = list(set(lst1) & set(lst2))
# print( intersection ) # ['C', 'D']
# # 차집합
# x = set(lst1) - set(intersection)
# print(list(x))


In [443]:
def checkMembers(post, curMembers):
    driver.get(post)

    wait.until(lambda d: d.find_element_by_xpath('//*[@id="ct"]/div[4]/div[3]/div/div[1]/a'))
    time.sleep(0.5)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    driver.find_element_by_xpath('//*[@id="ct"]/div[4]/div[3]/div/div[1]/a').send_keys(Keys.ENTER)

    wait.until(lambda d: d.find_elements_by_xpath('//*[@id="ct"]/div[4]/div/ul/li'))
    # 스크롤 다운
    last_height=0
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # 1초 대기
        time.sleep(1)

        # 스크롤 다운 후 스크롤 높이 다시 가져옴
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    members = driver.find_elements_by_xpath('//*[@id="ct"]/div[4]/div/ul/li')
    memberList = []
    for i in members:
        member = i.text.split('\n')[0]
        # print(member)
        memberList.append(member)

    # print(memberList)
    setMemberList = set(memberList)
    setCurMembers = set(curMembers)
    print('setMemberList : ', setMemberList)
    print('setCurMembers : ', setCurMembers)
    # 교집합
    intersection = set(memberList) & set(curMembers)
    # print( intersection ) # ['C', 'D']
    # 차집합
    x = set(curMembers) - intersection
    # print(list(x))
    print(x)
    return post, x

In [444]:
wbLoad = openpyxl.load_workbook('targetList.xlsx')
datas = wbLoad["Sheet1"]
print(datas)
posts = []
members = []
for i in datas:
    if i[0].value == '포스트유저' or i[1].value == '포스트 주소' or i[1].value == '검토멤버' :
        pass
    else :
        try :
            i[1].value = (i[1].value).replace('/blog.naver.com/', '/m.blog.naver.com/')
            if 'naver' in i[1].value:
                posts.append(i[1].value)
        except:
            pass
        if i[2].value is not None:
            members.append(i[2].value)
        # print("x : ", x)
        print("value : ", i[1].value)
print(posts)
print(members)

<Worksheet "Sheet1">
value :  None
value :  https://m.blog.naver.com/si_0225/222043558993
value :  https://m.blog.naver.com/si_0225/222046010461
value :  None
value :  None
value :  None
['https://m.blog.naver.com/si_0225/222043558993', 'https://m.blog.naver.com/si_0225/222046010461']
['라즈베리', '곱단', '앵니', '고블리스펙', '테스터']


In [445]:
finalChart = {}
for post in posts:
    x, y = checkMembers(post, members)
    print(
    f'''
    targetPost : {x}
    badMember(s) : {y}
    ''')
    for i in y:
        if i in finalChart.keys():
            finalChart[i]["cnt"] = finalChart[i]["cnt"]+1
            finalChart[i]["posts"].append(x)
        else :
            finalChart[i] = {"cnt":1, "posts":[x] }

    print(finalChart)

setMemberList :  {'핑크버블', '베네맘', '곱단', '웅비', '룽지언니', '도부장', 'REWHEEL', '부천공장사랑부동산', '일상적인 먹방', '발이바게트', '뇽', 'chemicalboys', '홀로그램', '지니', '안녕', '하수구뚫는명인', '놀기대장', '아띠홈', '은꽁', '로즈언니', '딜라잇', '조아', '선보', '라즈베리', '고미', '나나의봄날', '킴언니', '뷰스타 달콤달', '탕탕', '써니블리', '영또야', '고기싸롱', '미혜', '나리', '술의 종', '하면해', '소르소르', '컴파스', '랴', '킴치', '짱짱이', '또', '캠퍼꾸', '멋쟁이희준', '옝니', '데일리서우', '비니버디', 'kinkikids51', 'times0205', '홍당무', '초록뱀', '타니빵야', '몽몽', '정우디', '땡삐맘', '토낑이', '사라', '행복한이야기', '애송이줌관', '쉐프', 'CoCo SIRI', '이루리핏 EMS트레이닝', '홍나물', '달봉', '미미꽃', '정민', '쪼꼬쟁이', '쎄라킴', '이야이야홍홍', '해사한', '레몬버베나', '디해', '허스키보이스', '탬쓰', '흠냐링', '먕이', '마음이좋은사람', '지노', 'Grrrrrr', '테라', '쿠우', '이추', '제나팍', '근우준우맘', '고블리스펙트', '하미', '오늘민스', '김동화', '유니스', '곰곰이', '이쁜팅이', '해쯔니', '컴수리', '또링', '솜블리', '메이제이', '끼익', '이루다', '솔찡슬찡', '리샤', '망고노트', '팔오팔칠', '윤쏭쏭', '달떵이', '파나', '멜번가이', '요리요리', '쭈야예요', '세아', '율러블리', '여행은즐거워', '루월', '내마음의풍금', '리보니', '더플로지엔느', '경단기Supporter', '김마루씨', '정열매', '찰떡0430', '박새댁', '스텔라떼', '루시', '바게트', '신일주류상사'}
setCurMemb

In [446]:
def AutoFitColumnSize(worksheet, columns=None, margin=2):
    for i, column_cells in enumerate(worksheet.columns):
        is_ok = False
        if columns == None:
            is_ok = True
        elif isinstance(columns, list) and i in columns:
            is_ok = True

        if is_ok:
            length = max(len(str(cell.value)) for cell in column_cells)
            worksheet.column_dimensions[column_cells[0].column_letter].width = length + margin

    return worksheet

wbSave = openpyxl.Workbook()
thin = Side(border_style="thin", color="000000")
medium = Side(border_style="medium", color="000000")
double = Side(border_style="double", color="000000")
wbSaveSheet = wbSave.active
rowCnt = 2
wbSaveSheet[f"b{rowCnt}"] = "멤버이름"
wbSaveSheet[f"b{rowCnt}"].border = Border(top=double, left=double, right=thin, bottom=double)
wbSaveSheet[f"c{rowCnt}"] = "누락수"
wbSaveSheet[f"c{rowCnt}"].border = Border(top=double, left=thin, right=thin, bottom=double)
wbSaveSheet[f"d{rowCnt}"] = "포스트주소"
wbSaveSheet[f"d{rowCnt}"].border = Border(top=double, left=thin, right=double, bottom=double)
for member in finalChart:
    print('processing...')
    wbSaveSheet[f"b{rowCnt+1}"] = member
    wbSaveSheet[f"b{rowCnt+1}"].fill = PatternFill("solid", fgColor="00FFFF00")
    wbSaveSheet[f"b{rowCnt+1}"].border = Border(top=thin, left=double, right=thin, bottom=thin)
    wbSaveSheet[f"c{rowCnt+1}"] = finalChart[member]["cnt"]
    wbSaveSheet[f"c{rowCnt+1}"].border = Border(top=thin, left=thin, right=thin, bottom=thin)
    wbSaveSheet[f"d{rowCnt+1}"] = str(finalChart[member]["posts"])
    wbSaveSheet[f"d{rowCnt+1}"].border = Border(top=thin, left=thin, right=double, bottom=thin)
    rowCnt += 1
    print(
        f'''
        member : {member}
        Missing : {str(finalChart[member]["cnt"])}
        posts : {str(finalChart[member]["posts"])}
        ''')
wbSaveSheet[f"b{rowCnt+1}"].border = Border(top=double)
wbSaveSheet[f"c{rowCnt+1}"].border = Border(top=double)
wbSaveSheet[f"d{rowCnt+1}"].border = Border(top=double)
wbSaveSheet = AutoFitColumnSize(wbSaveSheet, margin=4)
wbSave.save('results.xlsx')




processing...

        member : 테스터
        Missing : 2
        posts : ['https://m.blog.naver.com/si_0225/222043558993', 'https://m.blog.naver.com/si_0225/222046010461']
        
processing...

        member : 앵니
        Missing : 2
        posts : ['https://m.blog.naver.com/si_0225/222043558993', 'https://m.blog.naver.com/si_0225/222046010461']
        
processing...

        member : 고블리스펙
        Missing : 2
        posts : ['https://m.blog.naver.com/si_0225/222043558993', 'https://m.blog.naver.com/si_0225/222046010461']
        


In [6]:
post = 'https://in.naver.com/chs3202/contents/258039385097760?query=%ED%83%88%EB%AA%A8%EC%83%B4%ED%91%B8'
driver.get(post)

In [7]:
# iframe = driver.find_elements_by_tag_name('iframe')[0]
# driver.switch_to_frame(iframe)
driver.switch_to.frame(driver.find_elements_by_tag_name('iframe')[0])
driver.find_element_by_class_name('btn_like_more').send_keys(Keys.ENTER)
# wait.until(lambda d: d.find_element_by_class_name('btn_like_more'))
# #body > div.floating_menu > div > div.btn_like_w > a
# //*[@id="body"]/div[4]/div[1]/h2/div/span[2]/a
# aaa = driver.find_elements_by_xpath('//*[@id="app"]/div[1]/div/div/div[2]/iframe')
# driver.switch_to_frame(aaa)
# driver.find_element_by_xpath('//*[@id="body"]/div[4]').send_keys(Keys.ENTER)
# driver.find_element_by_class_name('btn_like_more').send_keys(Keys.ENTER)
# driver.find_element_by_css_selector("#body > div.floating_menu > div > div.btn_like_w > a").send_keys(Keys.ENTER)